In [19]:
! pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 2.9 MB/s eta 0:00:00


In [20]:
from google.colab import userdata
TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')

In [ ]:
from together import Together

client = Together(api_key=TOGETHER_API_KEY)

In [ ]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1",
    messages=[
        {
                "role": "user",
                "content": "what is 1+1"
        },
        {
                "role": "assistant",
                "content": "<think>\nOkay, so the user is asking what 1 plus 1 is. That seems really simple, but maybe they‛re just starting to learn math. Let me think.\n\nFirst, I know that addition is combining two numbers. So if you have one apple and someone gives you another apple, you have two apples. That makes sense. But wait, is there any situation where 1+1 doesn‛t equal 2? Like in binary or different number systems? Hmm, in binary, 1+1 is 10, but the question doesn‛t specify the base. But usually, unless stated otherwise, we assume base 10. So in base 10, 1+1 is definitely 2. \n\nMaybe the user is testing me to see if I can explain it clearly. Let me make sure there‛s no trick here. Sometimes people use jokes or riddles where 1+1 could be something else, like in a partnership making 11, but that‛s not mathematical. The question is straightforward, so the answer should be straightforward too. \n\nI should also consider if there‛s any deeper context. If this is for a child learning addition, the answer is 2. If it‛s for a more advanced math concept, maybe there‛s a different angle, but the question doesn‛t indicate that. \n\nAlright, I think the best answer is 2. Just to double-check, I can count on my fingers. One finger plus another finger is two fingers. Yep, that‛s right. So I‛m confident the answer is 2.\n</think>\n\nThe result of 1 + 1 is **2**. This is a fundamental arithmetic operation where adding one unit to another unit gives a total of two units."
        }
    ],
    max_tokens=1024,
    temperature=0.7,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1,
    stop=["<｜end▁of▁sentence｜>"],
    stream=True
)


In [ ]:
for token in response:
    if hasattr(token, 'choices'):
        print(token.choices[0].delta.content, end='', flush=True)

In [ ]:
token.choices[0].delta.content

In [ ]:
answer = ''
for token in response:
    if hasattr(token, 'choices'):
        try:
            answer += token.choices[0].delta.content
        except:
            pass

In [ ]:
answer

'\n\n**Breakdown:**\n- **Base 10 (Decimal):** In the standard number system, 1 + 1 = 2.\n- **Binary (Base 2):** 1 + 1 = 10 (which represents the decimal number 2).\n- **Set Theory:** Combining a set with 1 element and another set with 1 element results in a set with 2 elements.\n\nNo hidden tricks or contexts are implied here—just straightforward addition! 🧮'

In [23]:
from together import Together

class ChatBot:
    """
    A simple ChatBot class to interact with a Together LLM model.

    Attributes:
        api_key (str): The API key used to authenticate with the Together API.
        client (Together): A Together client for making requests.
        history (list[dict]): A list of dictionaries representing the conversation history.
    """

    def __init__(self, api_key: str) -> None:
        """
        Initializes the ChatBot with a given API key and an empty conversation history.
        Also creates a Together client instance for making requests.

        Args:
            api_key (str): The API key for Together.
        """
        self.api_key: str = api_key
        self.client: Together = Together(api_key=self.api_key)
        self.history: list[dict] = []

    def append_history(self, role: str, content: str) -> None:
        """
        Appends a new message entry to the conversation history.

        Args:
            role (str): The role of the message sender, e.g., "user" or "assistant".
            content (str): The message content to be appended.
        """
        self.history.append({"role": role, "content": content})

    def invoke_api(
        self,
        model: str = "deepseek-ai/DeepSeek-V3",
        max_tokens: int = 1024,
        temperature: float = 0.7,
        top_p: float = 0.7,
        top_k: int = 50,
        repetition_penalty: float = 1.0,
        stop: list[str] = ["<｜end▁of▁sentence｜>"]
    ) -> str:
        """
        Invokes the Together chat API using the stored conversation history.

        Args:
            model (str, optional): The name of the Together model to use. Defaults to "deepseek-ai/DeepSeek-V3".
            max_tokens (int, optional): The maximum number of tokens in the response. Defaults to 1024.
            temperature (float, optional): The sampling temperature. Defaults to 0.7.
            top_p (float, optional): The top_p sampling parameter. Defaults to 0.7.
            top_k (int, optional): The top_k sampling parameter. Defaults to 50.
            repetition_penalty (float, optional): The repetition penalty parameter. Defaults to 1.0.
            stop (list[str], optional): A list of stop tokens. Defaults to ["<｜end▁of▁sentence｜>"].

        Returns:
            str: The collapsed string response from the API.
        """
        response = self.client.chat.completions.create(
            model=model,
            messages=self.history,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            stop=stop,
            stream=True
        )
        answer: str = self.collapse_response(response)
        return answer

    def collapse_response(self, response) -> str:
        """
        Collapses a streaming response from the Together API into a single string.

        Args:
            response: The streaming response object from the Together API.

        Returns:
            str: A single string containing the concatenated content from each token in the response.
        """
        answer: str = ""
        for token in response:
            if hasattr(token, "choices"):
                try:
                    answer += token.choices[0].delta.content
                except:
                    pass
        return answer

    def show_history(self) -> None:
        """
        Prints the entire conversation history.
        """
        print(self.history)


In [26]:
# Instantiate the ChatBot
bot = ChatBot(api_key=TOGETHER_API_KEY)

bot.history = [{"role": "assistant", "content": "You always provide reasoning. Your answer starts from <think>xxx</think> and <response>."}]

# Provided by data
current_question = "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"
current_answer = "72"
augmented_content = f"Provide reasoning how to answer question: {current_question} and to arrive with answer: {current_answer}"
print(augmented_content)

# Append augmented content to history
bot.append_history(role="user", content=augmented_content)
bot.invoke_api()

Provide reasoning how to answer question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? and to arrive with answer: 72


"<think>\nTo solve the problem, we need to determine the total number of clips Natalia sold in April and May. Here's the step-by-step reasoning:\n\n1. **Clips sold in April**: Natalia sold clips to 48 friends in April. So, the number of clips sold in April is **48**.\n\n2. **Clips sold in May**: In May, she sold half as many clips as she did in April. To find this, we calculate half of 48:\n   \\[\n   \\text{Clips sold in May} = \\frac{48}{2} = 24\n   \\]\n\n3. **Total clips sold**: To find the total number of clips sold in April and May, we add the number of clips sold in each month:\n   \\[\n   \\text{Total clips} = \\text{Clips sold in April} + \\text{Clips sold in May} = 48 + 24 = 72\n   \\]\n\nThus, Natalia sold a total of **72 clips** in April and May.\n</think>\n\n<response>\n72\n</response>"

In [ ]:
import os

# Replace with your actual Together API key
# TOGETHER_API_KEY = "YOUR_TOGETHER_API_KEY"

# Instantiate the ChatBot
bot = ChatBot(api_key=TOGETHER_API_KEY)
print("Welcome to the ChatBot! Type 'exit' or 'quit' to end the conversation.\n")

while True:
    try:
        # Get user input
        user_input = input("🧑‍💻 You: ")

        # Check for exit condition
        if user_input.strip().lower() in ["exit", "quit"]:
            print("👋 Ending the conversation. Goodbye!")
            break

        # Append user message to history
        bot.append_history(role="user", content=user_input)

        # Invoke the API to get the assistant's response
        assistant_response = bot.invoke_api()

        # Append assistant response to history
        bot.append_history(role="assistant", content=assistant_response)

        # Display the assistant's response with emoji
        print(f"🤖 Assistant: {assistant_response}\n")

    except KeyboardInterrupt:
        print("\n👋 Conversation interrupted. Goodbye!")
        break
    except Exception as e:
        print(f"❌ An error occurred: {e}")


Welcome to the ChatBot! Type 'exit' or 'quit' to end the conversation.

🧑‍💻 You: what is 1+1
🤖 Assistant: <think>
Okay, the user is asking "what is 1+1". Let me think about how to approach this.

First, I need to make sure I understand the question correctly. They're asking for the sum of 1 and 1. That seems straightforward, but maybe they want a more detailed explanation, especially if they're just learning basic arithmetic.

So, 1 plus 1. In basic addition, when you add two numbers, you're combining their values. Here, combining one unit with another unit gives you two units. So, 1 + 1 equals 2. 

But wait, maybe I should check if there's any context I'm missing. Sometimes people ask simple questions as a test or a joke. But the user hasn't given any indication of that. They might be a student starting out with math, so a clear and simple answer is best.

I should also consider different ways to explain it. For example, using objects: if you have one apple and someone gives you anoth

In [2]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.7 MB/s eta 0:00:00


In [9]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

In [30]:
# Load the full dataset
data = load_dataset('openai/gsm8k', 'main')

# Select only the first 100 samples
data_small = data['train'].select(range(100))

# Print dataset summary
print(data_small)

Dataset({
    features: ['question', 'answer'],
    num_rows: 100
})


In [52]:
data

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})

In [53]:
data = data['train']

In [17]:
data_small = data_small['train']

In [31]:
data_small

Dataset({
    features: ['question', 'answer'],
    num_rows: 100
})

In [32]:
from datasets import Dataset
from tqdm import tqdm  # For progress tracking

# Function to augment an answer using the ChatBot
def augment_answer(example):
    """
    Takes a dictionary containing a 'question' and 'answer',
    creates a new ChatBot instance with a clean history,
    calls the ChatBot, and replaces the answer with the augmented response.
    """
    # Instantiate a new ChatBot (clean slate)
    bot = ChatBot(api_key=TOGETHER_API_KEY)

    # Set up initial chatbot history
    bot.history = [
        {"role": "assistant", "content": "You always provide reasoning. Your answer starts from <think>xxx</think> and <response>."}
    ]

    current_question = example["question"]
    current_answer = example["answer"]

    # Construct the augmented content
    augmented_content = f"Provide reasoning how to answer question: {current_question} and to arrive with answer: {current_answer}"

    # Append question to chatbot history
    bot.append_history(role="user", content=augmented_content)

    # Call API and get response
    response = bot.invoke_api()

    # Update the answer field with the model's response
    example["answer"] = response  # Assign bot output to 'answer'

    return example

In [ ]:
%%time

# Apply the function to each row in the dataset
data = data.map(augment_answer, num_proc=4)  # Parallel processing for speed

# Save the updated dataset (optional)
data.save_to_disk("augmented_gsm8k")

Map (num_proc=4):   0%|          | 0/7473 [00:00<?, ? examples/s]

ChunkedEncodingError: Response ended prematurely

In [ ]:
# Save the updated dataset (optional)
data.save_to_disk("augmented_gsm8k_train")

In [ ]:
data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore

In [33]:
%%time

# Apply the function to each row in the dataset
aug_data = data_small.map(augment_answer, num_proc=4)  # Parallel processing for speed

# Save the updated dataset (optional)
aug_data.save_to_disk("augmented_gsm8k")

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

CPU times: user 7.29 s, sys: 1.11 s, total: 8.41 s
Wall time: 16min 47s


In [40]:
aug_data['answer'][0]

"<think>  \nTo answer the question, we need to determine the total number of clips Natalia sold in April and May. Here's the reasoning process:\n\n1. **Clips sold in April**: The question states that Natalia sold clips to 48 of her friends in April. So, the number of clips sold in April is 48.\n\n2. **Clips sold in May**: The question mentions that Natalia sold half as many clips in May as she did in April. To find this, we divide the number of clips sold in April by 2.  \n   Calculation: \\( 48 \\div 2 = 24 \\).  \n   So, Natalia sold 24 clips in May.\n\n3. **Total clips sold in April and May**: To find the total, we add the number of clips sold in April and May.  \n   Calculation: \\( 48 + 24 = 72 \\).  \n   Therefore, Natalia sold a total of 72 clips in April and May.\n\nThe final answer is 72.  \n</think>  \n<response>  \n#### 72  \n</response>"

In [41]:
aug_data = aug_data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [46]:
aug_data

Dataset({
    features: ['question', 'answer', 'prompt'],
    num_rows: 100
})

In [48]:
aug_data[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': '72  \n</response>',
 'prompt': [{'content': '\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n',
   'role': 'system'},
  {'content': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
   'role': 'user'}]}

In [51]:
from datasets import load_from_disk, DatasetDict

# Load dataset from the saved folder
aug_data = load_from_disk("augmented_gsm8k_train")

# Convert it into a DatasetDict format
dataset_dict = DatasetDict({"train": aug_data})

# Save again in correct structure
dataset_dict.save_to_disk("augmented_gsm8k_train")

# Define user and repo details
user_name = "eagle0504"
repo_name = "openai-gsm8k-augmented-using-together-ai-deepseek-v3-train-enhanced"  # Change this to your desired repository name

# Push to Hugging Face
dataset_dict.push_to_hub(f"{user_name}/{repo_name}")

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/eagle0504/openai-gsm8k-augmented-using-together-ai-deepseek-v3-small/commit/d06df399bbcd0458381530293128f27a6c3e2c10', commit_message='Upload dataset', commit_description='', oid='d06df399bbcd0458381530293128f27a6c3e2c10', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/eagle0504/openai-gsm8k-augmented-using-together-ai-deepseek-v3-small', endpoint='https://huggingface.co', repo_type='dataset', repo_id='eagle0504/openai-gsm8k-augmented-using-together-ai-deepseek-v3-small'), pr_revision=None, pr_num=None)